In [1]:
%run utils.py

In [89]:
test = """\
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

# count those that have all required fields. Treat cid as optional.
key_set = {'byr', 'cid', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
opt_key = {'cid'}
regex_input = "(\w+):([a-zA-Z0-9#]+)"

def test_reg(text:str):
    for s in text.split('\n\n'):
        yield re.findall(regex_input,s)

list(test_reg(test))

[[('ecl', 'gry'),
  ('pid', '860033327'),
  ('eyr', '2020'),
  ('hcl', '#fffffd'),
  ('byr', '1937'),
  ('iyr', '2017'),
  ('cid', '147'),
  ('hgt', '183cm')],
 [('iyr', '2013'),
  ('ecl', 'amb'),
  ('cid', '350'),
  ('eyr', '2023'),
  ('pid', '028048884'),
  ('hcl', '#cfa07d'),
  ('byr', '1929')],
 [('hcl', '#ae17e1'),
  ('iyr', '2013'),
  ('eyr', '2024'),
  ('ecl', 'brn'),
  ('pid', '760753108'),
  ('byr', '1931'),
  ('hgt', '179cm')],
 [('hcl', '#cfa07d'),
  ('eyr', '2025'),
  ('pid', '166559648'),
  ('iyr', '2011'),
  ('ecl', 'brn'),
  ('hgt', '59in')]]

In [97]:
def tok(iter_in):
    def mk_key_set(kv_tuple_list):
        d = dict()
        for (k,v) in kv_tuple_list:
            d[k]=v
        return set(d.keys())
    
    def valid_key_set(s):
        return 1 if s == key_set or s == key_set-opt_key else 0
    
    accum = []
    for line in iter_in:
        l = re.findall(regex_input,line)
        print(l)
        if len(l) :
            accum += l
        else :
            yield valid_key_set(mk_key_set(accum))
            accum = []
    # get last accumulated
    d = dict()
    for (k,v) in accum:
        d[k]=v
    yield 1 if set(d.keys()) == key_set or set(d.keys()) == key_set-opt_key else 0
            

sum(list(tok(test.splitlines())))

[('ecl', 'gry'), ('pid', '860033327'), ('eyr', '2020'), ('hcl', '#fffffd')]
[('byr', '1937'), ('iyr', '2017'), ('cid', '147'), ('hgt', '183cm')]
[]
[('iyr', '2013'), ('ecl', 'amb'), ('cid', '350'), ('eyr', '2023'), ('pid', '028048884')]
[('hcl', '#cfa07d'), ('byr', '1929')]
[]
[('hcl', '#ae17e1'), ('iyr', '2013')]
[('eyr', '2024')]
[('ecl', 'brn'), ('pid', '760753108'), ('byr', '1931')]
[('hgt', '179cm')]
[]
[('hcl', '#cfa07d'), ('eyr', '2025'), ('pid', '166559648')]
[('iyr', '2011'), ('ecl', 'brn'), ('hgt', '59in')]


2

In [95]:
sum(list(tok(open("input/input4.txt"))))

228

In [75]:
set(data.keys()) == key_set

True

In [49]:
"".join([data,"cazzo"])

'cazzocazzo'